**Covid-19 Crisis**


Copyright 2020. Kiarash Kiani.
* **Home Page:**    http://kiarash.info
* **Email:**        hi@kiarash.info
* **GitHub:**       github.com/kkiani

# Introduction
Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified in 2019 in Wuhan, China, and has since spread globally, resulting in the 2019–20 coronavirus pandemic. According to the World Health Organization (WHO), the most common symptoms are fever, tiredness, and dry cough.

[Read more on Wikipedia](https://en.wikipedia.org/wiki/Coronavirus_disease_2019)

# Dataset
This dataset provided by [Our World in Data](https://ourworldindata.org/coronavirus-source-data) website. In the first steps we do some basic Pandas manipulation inorder to get know the nature of our data better.

<span style="background-color: red;">*ATTENTION: This notebook created by the time that dataset has only recoreds until 15 march 2020.*</span>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [ ]:
total_cases_path = 'https://covid.ourworldindata.org/data/ecdc/total_cases.csv'
total_deaths_path = 'https://covid.ourworldindata.org/data/ecdc/total_deaths.csv'
new_cases_path = 'https://covid.ourworldindata.org/data/ecdc/new_cases.csv'

df = pd.read_csv(total_cases_path)

List of countries

In [ ]:
df.columns.values

5 most affected countries:

In [ ]:
last_row  = df.tail(1).copy()
last_row.drop(['World', 'date'], axis=1, inplace=True)

last_row = last_row.T
last_row.reset_index()
last_row.columns = ['total cases']

In [ ]:
last_row.sort_values(['total cases'], ascending=False).head(5)

5 less affected countries

In [ ]:
last_row.sort_values(['total cases'], ascending=False).tail(5)

How many days we pass from the begining of crisis?

In [ ]:
months = df.shape[0]//30
days = df.shape[0]%30

print(f'{months} months and {days} days. {df.shape[0]} days in total.')

Plot growing of cases in the 5 most affected contries

In [ ]:
china = df['China'].fillna(0).values
italy = df['Italy'].fillna(0).values
iran = df['Iran'].fillna(0).values
korea = df['South Korea'].fillna(0).values
spain = df['Spain'].fillna(0).values
world = df['World'].fillna(0).values

plt.plot(china, label='china')
plt.plot(italy, label='italy')
plt.plot(iran, label='iran')
plt.plot(korea, label='south korea')
plt.plot(spain, label='spain')
plt.plot(world, label='world')

plt.legend()
plt.show()

Looks like China has a diffrent pattern compared to other four countries. let remove china

In [ ]:
world_without_china = world - china

plt.plot(italy, label='italy')
plt.plot(iran, label='iran')
plt.plot(korea, label='south korea')
plt.plot(spain, label='spain')
plt.plot(world_without_china, label='world without china')

plt.legend()
plt.show()

In [ ]:
iran = df['Iran'].fillna(0).values

plt.plot(iran, label='Iran')
plt.plot(world_without_china, label='world without china')
plt.legend()
plt.show()

Plot Iran growing number of cases from the beging of Global Crises

In [ ]:
plt.plot(iran)
plt.show()

In [ ]:
iran2 = df['Iran'].dropna().values

plt.plot(iran2)
plt.show()

In [ ]:
italia = df['Italy'].fillna(0).values

plt.plot(iran, label='Iran')
plt.plot(italia, label='Italy')
#plt.plot(other_than_china, label='world without china')
plt.legend()
plt.show()

In [ ]:
iran = df['Iran'].fillna(1).values
italy = df['Italy'].fillna(1).values

plt.plot(np.log(iran), label='iran')
plt.plot(np.log(italy), label='italy')
plt.legend()
plt.show()

In [ ]:
korea = df['South Korea'].fillna(0).values

plt.plot(korea)
plt.show()